# Data Engineering your stocks

In [64]:
import os
import pandas as pd
import numpy as np
import time

# Task 1: Get stock price data from API

In [42]:
def get_stock_prices(stock: str, apikey: str):
    api_result = get_stock_API(stock, apikey)
    output = transform_api_output(api_result)
    output['ticker'] = stock
    output['close'] = output['close'].astype(float)
    return output

In [43]:
import requests
def get_stock_API(stock: str, apikey: str):
    API_URL = "https://www.alphavantage.co/query"
    data = {
        "function": "TIME_SERIES_DAILY",
        "symbol": stock,
        "outputsize": "compact",
        "datatype": "csv",
        "apikey": apikey,
    }
    response = requests.get(API_URL, data)
    if 'Invalid API call' in response.text:
        raise KeyError("Ticker not valid!")
    while '5 calls per minute' in response.text:
        time.sleep(60)
        response = requests.get(API_URL, data)
    return response.text

In [37]:
api_result = get_stock_API('AAPL', os.environ['API_KEY'])
api_result

'timestamp,open,high,low,close,volume\r\n2022-10-21,142.8700,147.8479,142.6490,147.2700,86548609\r\n2022-10-20,143.0200,145.8900,142.6500,143.3900,64521989\r\n2022-10-19,141.6900,144.9492,141.5000,143.8600,61758340\r\n2022-10-18,145.4900,146.7000,140.6100,143.7500,99136610\r\n2022-10-17,141.0650,142.9000,140.2700,142.4100,85250939\r\n2022-10-14,144.3100,144.5200,138.1900,138.3800,88597969\r\n2022-10-13,134.9900,143.5900,134.3700,142.9900,113223975\r\n2022-10-12,139.1300,140.3600,138.1600,138.3400,70433744\r\n2022-10-11,139.9000,141.3500,138.2200,138.9800,77033672\r\n2022-10-10,140.4200,141.8900,138.5729,140.4200,74899002\r\n2022-10-07,142.5400,143.1000,139.4450,140.0900,85925559\r\n2022-10-06,145.8100,147.5400,145.2200,145.4300,68402169\r\n2022-10-05,144.0750,147.3800,143.0100,146.4000,79470968\r\n2022-10-04,145.0300,146.2200,144.2600,146.1000,87830064\r\n2022-10-03,138.2100,143.0700,137.6850,142.4500,114311663\r\n2022-09-30,141.2800,143.1000,138.0000,138.2000,124925274\r\n2022-09-29,1

### Your code

In [68]:
def transform_api_output(api_result: str):
    return pd.DataFrame()

Test your code

In [47]:
result = transform_api_output(api_result)
print(f"Check 1: Actual shape: {result.shape}, expected shape: {(100,6)}")
print(f"Check 2: Actual columns: {list(result.columns)}, expected columns: {['timestamp', 'open', 'high', 'low', 'close', 'volume']}")

Check 1: Actual shape: (100, 6), expected shape: (100, 6)
Check 2: Actual columns: ['timestamp', 'open', 'high', 'low', 'close', 'volume'], expected columns: ['timestamp', 'open', 'high', 'low', 'close', 'volume']


Check [Dashboard](http://127.0.0.1:8505)

# Task 2: Buffering Data

In [57]:
def get_or_load(stock: str, apikey: str):
    # Define folder, filename and file path
    file_name = f'{stock}.csv'
    data_dir = 'data/current/'
    file_path = f'{data_dir}{file_name}'
    
    # Check if stock already in data lake
    if file_name in os.listdir(data_dir):
        data = pd.read_csv(file_path, index_col=0)
        return data
    
    # If stock not found locally, return API call
    data = get_stock_prices(stock, apikey)
    if 'timestamp' in data.columns:
        data.to_csv(file_path)
        return data
    else:
        raise KeyError('Column timestamp not in dataframe!')
        
    

In [50]:
get_or_load('AAPL', os.environ['API_KEY'])

,timestamp,open,high,low,close,volume,ticker
0,2022-10-21,142.870,147.8479,142.649,147.27,86548609,AAPL
1,2022-10-20,143.020,145.8900,142.650,143.39,64521989,AAPL
2,2022-10-19,141.690,144.9492,141.500,143.86,61758340,AAPL
3,2022-10-18,145.490,146.7000,140.610,143.75,99136610,AAPL
4,2022-10-17,141.065,142.9000,140.270,142.41,85250939,AAPL
...,...,...,...,...,...,...,...
95,2022-06-07,144.345,149.0000,144.100,148.71,67808150,AAPL
96,2022-06-06,147.030,148.5689,144.900,146.14,71598380,AAPL
97,2022-06-03,146.900,147.9700,144.460,145.38,88570289,AAPL
98,2022-06-02,147.830,151.2700,146.860,151.21,72348055,AAPL


### Your code

# Task 3: Get and clean historical data

In [63]:
from utils import get_historical_data
def get_stock_timeline(stock: str, apikey: str):
    current_data = get_or_load(stock, apikey)
    
    hist_data_raw = get_historical_data(stock, apikey)
    # TODO: define transformation function for historical data
    hist_data = transform_hist_data(hist_data_raw)
    
    output = pd.concat([current_data,hist_data])
    output['close'] = output['close'].astype(float)
    return output

In [21]:
historical_prices = get_historical_data('AAPL', os.environ['API_KEY'])
historical_prices

,timestamp,open,high,low,close,volume,ticker
100,2022/05/31,149.0700,150.6600,146.8400,14884.0,103718416,AAPL
101,2022/05/27,145.3900,149.6800,145.2600,14964.0,90978503,AAPL
102,2022/05/26,137.3900,144.3400,137.1400,14378.0,90601548,AAPL
103,2022/05/25,138.4300,141.7850,138.3400,14052.0,92482696,AAPL
104,2022/05/24,140.8050,141.9700,137.3300,14036.0,104132746,AAPL
...,...,...,...,...,...,...,...
5777,1999/11/05,84.6200,88.3700,84.0000,8831.0,3721500,AAPL
5778,1999/11/04,82.0600,85.3700,80.6200,8362.0,3384700,AAPL
5779,1999/11/03,81.6200,83.2500,81.0000,8150.0,2932700,AAPL
5780,1999/11/02,78.0000,81.6900,77.3100,8025.0,3564600,AAPL


In [51]:
get_or_load('AAPL', os.environ['API_KEY'])

,Unnamed: 0,timestamp,open,high,low,close,volume,ticker
0,0,2022-10-21,142.870,147.8479,142.649,147.27,86548609,AAPL
1,1,2022-10-20,143.020,145.8900,142.650,143.39,64521989,AAPL
2,2,2022-10-19,141.690,144.9492,141.500,143.86,61758340,AAPL
3,3,2022-10-18,145.490,146.7000,140.610,143.75,99136610,AAPL
4,4,2022-10-17,141.065,142.9000,140.270,142.41,85250939,AAPL
...,...,...,...,...,...,...,...,...
95,95,2022-06-07,144.345,149.0000,144.100,148.71,67808150,AAPL
96,96,2022-06-06,147.030,148.5689,144.900,146.14,71598380,AAPL
97,97,2022-06-03,146.900,147.9700,144.460,145.38,88570289,AAPL
98,98,2022-06-02,147.830,151.2700,146.860,151.21,72348055,AAPL


### Your code

In [53]:
def transform_hist_data(df: pd.DataFrame):
    output = df
    return output

Test your code

In [54]:
result = transform_hist_data(historical_prices)
result_current = get_or_load('AAPL', os.environ['API_KEY'])
print(f"Check 1: Actual close mean: {result.close.mean()}, expected mean: ~{174}")
print(f"Check 2: Actual date format: {result.timestamp.to_list()[-1]}, expected date format: 1999-11-01")

Check 1: Actual close mean: 174.07027602956705, expected mean: ~174
Check 2: Actual date format: 1999-11-01, expected date format: 1999-11-01


Check [Dashboard](http://127.0.0.1:8505)

# Task 4: Get stock prices for multiple stocks

In [23]:
from pytickersymbols import PyTickerSymbols
def get_ticker_symbols():
    stock_data = PyTickerSymbols()
    stocks = stock_data.get_sp_100_nyc_yahoo_tickers()
    return stocks

In [24]:
get_ticker_symbols()[:10]

['LIN', 'MMM', 'AXP', 'AAPL', 'BA', 'CAT', 'CVX', 'CSCO', 'KO', 'DD']

In [ ]:
def get_stock_timeline(stock: str, apikey: str):
    current_data = get_or_load(stock, apikey)
    
    hist_data_raw = get_historical_data(stock, apikey)
    # TODO: define transformation function for historical data
    hist_data = transform_hist_data(hist_data_raw)
    
    output = pd.concat([current_data, hist_data])
    return output

### Your code

In [25]:
from typing import List 

def get_stocks(stocks: List[str], apikey: str) -> pd.DataFrame:
    # TODO: Expand the output to multiple stocks, use get_stock_timeline
    return pd.DataFrame()

Check 1: Actual stocks [], expected stocks: ['AAPL', 'TSLA']


Test your code

In [ ]:
result = get_stocks(['AAPL', 'TSLA'], os.environ['API_KEY'])
if 'ticker' in result.columns:
    ticker = list(result.ticker.unique())
else:
    ticker = []
print(f"Check 1: Actual stocks {ticker}, expected stocks: {['AAPL', 'TSLA']}")

Check [Dashboard](http://127.0.0.1:8505)

# Task 5: Feature Engineering
## Lets add additonal features such as stock performance

In [ ]:
get_or_load('AAPL', os.environ['API_KEY'])

### Your code

In [ ]:
def get_performance(df: pd.DataFrame):
    return df

Test your code

In [ ]:
result = get_performance(df)
print(f"Check 2: Actual columns: {list(result.columns)}, expected columns: {['timestamp', 'open', 'high', 'low', 'close', 'volume', 'ticker', 'performance']}")

Check [Dashboard](http://127.0.0.1:8505)